# The Data

In [2]:
# import required libraries
%run pre_req.py

In [3]:
raw_data = pd.read_csv('Churn_Modelling.csv')
raw_data.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6252,6253,15687492,Anderson,596,Germany,Male,32,3,96709.07,2,0,0,41788.37,0
4684,4685,15736963,Herring,623,France,Male,43,1,0.00,2,1,1,146379.30,0
1731,1732,15721730,Amechi,601,Spain,Female,44,4,0.00,2,1,0,58561.31,0
4742,4743,15762134,Liang,506,Germany,Male,59,8,119152.10,2,1,1,170679.74,0
4521,4522,15648898,Chuang,560,Spain,Female,27,7,124995.98,1,1,1,114669.79,0


In [4]:
raw_data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
df = raw_data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6366,683,Germany,Male,73,9,124730.26,1,1,1,51999.50,0
9628,662,France,Female,22,9,0.00,2,1,1,44377.65,0
5916,633,Spain,Female,39,2,0.00,2,0,0,191207.03,0
681,591,Spain,Male,58,5,128468.69,1,0,1,137254.55,0
5459,530,Spain,Female,36,7,0.00,2,1,0,80619.09,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [7]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

## Object to Numeric

### Label Encoder

In [9]:
# fitting 
label_encoder_gender = LabelEncoder()
label_encoder_gender.fit(df['Gender'])
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [10]:
# applying
df_encoded = df.copy()
df_encoded['Gender'] = label_encoder_gender.transform(df_encoded['Gender'])
df_encoded.sample(5)['Gender']

8740    0
7755    1
1092    1
6635    0
2854    1
Name: Gender, dtype: int64

In [11]:
# saving the file
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

### OneHotEncoder

In [12]:
# FITTING
onehot_encoder_geography = OneHotEncoder()
onehot_encoder_geography.fit(df[['Geography']]) #DATAFRAME IS REQUIRED (2D)
onehot_encoder_geography.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [13]:
# Applying
encoded = onehot_encoder_geography.transform(df[['Geography']])
encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [14]:
onehot_encoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [15]:
encoded_df = pd.DataFrame(encoded.toarray(), columns = onehot_encoder_geography.get_feature_names_out(['Geography']))
encoded_df.sample(5)

,Geography_France,Geography_Germany,Geography_Spain
4449,1.0,0.0,0.0
1468,1.0,0.0,0.0
6973,0.0,1.0,0.0
6347,0.0,1.0,0.0
9000,1.0,0.0,0.0


In [16]:
df_encoded = pd.concat([df_encoded.drop(['Geography'], axis = 1), encoded_df], axis = 1)
df_encoded.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
8718,589,1,37,7,85146.48,2,1,0,86490.09,1,1.0,0.0,0.0
1765,766,1,43,6,112088.04,2,1,1,36706.56,0,0.0,1.0,0.0
3011,526,1,30,8,0.00,1,1,0,36251.00,0,0.0,0.0,1.0
4187,656,1,32,5,136963.12,1,1,0,133814.28,0,0.0,0.0,1.0
3431,468,1,36,3,61636.97,1,0,0,107787.42,0,1.0,0.0,0.0


In [17]:
# saving file
with open('onehot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geography, file)

## Scalling

In [18]:
X = df_encoded.drop(['Exited'], axis = 1)
y = df_encoded['Exited']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((8000, 12), (2000, 12))

In [20]:
# fitting
scaler = StandardScaler()
scaler.fit(X_train)
scaler

,copy,True
,with_mean,True
,with_std,True


In [21]:
# applying
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]], shape=(8000, 12))

In [22]:
# saving
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# Training

In [23]:
X_train.shape[1]

12

In [24]:
# modeling
model = Sequential([Dense(units = 64, input_shape = (X_train.shape[1],), activation = 'relu'),
                   Dense(units = 32, activation = 'relu'),
                   Dense(units = 1, activation = 'sigmoid')
                   ])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# loss function and optimizer
loss_func = BinaryCrossentropy()
optimizer = Adam(learning_rate = 0.01)

model.compile(loss = loss_func,
             optimizer = optimizer,
             metrics = ['Accuracy']
             )
model

<Sequential name=sequential, built=True>

In [26]:
# saving trainings dir
log_dir = "logs/fit/" + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
log_dir

'logs/fit/20251118_084017'

In [27]:
# callbacks
tensorflow_callback = TensorBoard(log_dir = log_dir,
                                 histogram_freq = 1
                                 )

earlystopping_callback = EarlyStopping(monitor = 'val_loss',
                                      patience = 5,
                                      restore_best_weights = True
                                      )

In [28]:
# training
history = model.fit(X_train, y_train,
               validation_data = (X_test, y_test),
               epochs = 100,
               callbacks = [tensorflow_callback, earlystopping_callback]
               )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - Accuracy: 0.8367 - loss: 0.3984 - val_Accuracy: 0.8545 - val_loss: 0.3540
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - Accuracy: 0.8544 - loss: 0.3574 - val_Accuracy: 0.8515 - val_loss: 0.3478
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - Accuracy: 0.8572 - loss: 0.3509 - val_Accuracy: 0.8570 - val_loss: 0.3437
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - Accuracy: 0.8579 - loss: 0.3430 - val_Accuracy: 0.8555 - val_loss: 0.3449
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - Accuracy: 0.8575 - loss: 0.3399 - val_Accuracy: 0.8555 - val_loss: 0.3493
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - Accuracy: 0.8610 - loss: 0.3376 - val_Accuracy: 0.8570 - val_loss: 0.3613
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - Accuracy: 0.8616 - loss: 0.3371 - val_Accuracy: 0.8625 - val_loss: 0.3467
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Accuracy: 0.8624 - loss: 0.3323 - val_Accu

In [29]:
# saving data model
model.save('model.keras')

In [42]:
%load_ext tensorboard
%tensorboard --logdir logs/fit --port 6007

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
